# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [10]:
# TO-DO: Create a Keyspace 
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS event_data 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

#### Set Keyspace

In [11]:
# TO-DO: Set KEYSPACE to the keyspace specified above
session.set_keyspace('event_data')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### Query 1: Display artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

#### Step 1 :  Creating the table artist_song_info

In [12]:
query = "CREATE TABLE IF NOT EXISTS artist_song_info "
query = query + "(sessionid int, item_number_session int, artist text, song_title text, song_length float, \
                     PRIMARY KEY (sessionid,item_number_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

                    

#### Step 2 : set up the CSV file and insert the data into the table created in music_library_1

In [13]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO artist_song_info \
                    (sessionid, item_number_session, artist, song_title, song_length)"
        query = query + " VALUES (%s, %s, %s, %s,%s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[8]), int(line[3]), line[0],line[9], float(line[5])))

#### Step 3 : Do a SELECT to verify that the data have been inserted into each table

* ##### For table artist_song_info, the sessionid column was used as a partition key because the queries will filter by this column. item_number_session were used as clustering columns to help make up a unique key and also it makes logically sense to sort the data with item_number_session(cluster_key).

In [21]:
query = "SELECT artist, song_title,song_length \
                from artist_song_info \
                    where sessionid = 338 and \
                        item_number_session = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

print("artistname","|", "songname","|", "songlength")
for row in rows:
    print (row.artist,"|", row.song_title,"|", row.song_length)

artistname | songname | songlength
Faithless | Music Matters (Mark Knight Dub) | 495.30731201171875


### Query 2: Display the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

#### Step 1 :  Creating the table artist_song_user_info

In [17]:
query = "CREATE TABLE IF NOT EXISTS artist_song_user_info "
query = query + "(sessionid int, userid int, item_number_session int, artist text, song_title text,  \
                    firstname text, lastname text,  \
                        PRIMARY KEY ((sessionid,userid),item_number_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

                    

#### Step 2 : set up the CSV file and insert the data into the table created in music_library_2

In [18]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO artist_song_user_info (sessionid, userid, \
                    item_number_session, artist, song_title, \
                        firstname, lastname)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[8]), int(line[10]),int(line[3]),  \
                                    line[9], line[0], line[1], \
                                        line[4]))

#### Step 3 : Do a SELECT to verify that the data have been inserted into each table

* ##### For table artist_song_user_info, the sessionid column and userid together were used as a partition key because the queries will filter by these 2 columns and item_number_session were used as clustering column to sort the data with this column.

In [22]:
query = "SELECT artist, song_title,  firstname, lastname \
            from artist_song_user_info \
                where sessionid = 182 and \
                    userid = 10"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
print("artistname","|", "songname","|", "userfirstName","|", "userLastName")   
for row in rows:
    print (row.artist,"|",  row.song_title,"|",  row.firstname,"|",  row.lastname) 

artistname | songname | userfirstName | userLastName
Keep On Keepin' On | Down To The Bone | Sylvie | Cruz
Greece 2000 | Three Drives | Sylvie | Cruz
Kilometer | Sebastien Tellier | Sylvie | Cruz
Catch You Baby (Steve Pitron & Max Sanna Radio Edit) | Lonnie Gordon | Sylvie | Cruz


### Query 3: Display user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

#### Step 1 :  Creating the table user_fullname_info_filtersong

In [23]:
query = "CREATE TABLE IF NOT EXISTS user_fullname_info_filtersong "
query = query + "(song_title text,userid int, firstname text, lastname text, \
                    PRIMARY KEY (song_title, userid))"
try:
    session.execute(query)
except Exception as e:
    print(e)


#### Step 2 : set up the CSV file and insert the data into the table created in music_library_3

In [24]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        #Assign the INSERT statements into the `query` variable
        query = "INSERT INTO user_fullname_info_filtersong \
                    (song_title, userid, firstname, lastname)"
        query = query + " VALUES (%s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (line[9], int(line[10]), line[1],line[4]))

#### Step 3 : Do a SELECT to verify that the data have been inserted into each table

* ##### For table user_fullname_info_filtersong, the song_title column was used as a partition key because the queries will filter by this column. userid column had to be included in this table even though it is not used to display in end result because, it would eliminate the use of ALLOW FILTERING condition (if we include firstname and last name as clustering key, it would degraade the performance of the query).

In [25]:
query = "SELECT firstname, lastname \
            from user_fullname_info_filtersong \
                where song_title = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

print("userfirstName","|", "userLastName")  
for row in rows:
    print (row.firstname,"|", row.lastname) 

userfirstName | userLastName
Jacqueline | Lynch
Tegan | Levine
Sara | Johnson


### Drop the tables before closing out the sessions

In [26]:
## Drop the table before closing out the sessions

In [27]:
query1 = "DROP TABLE IF EXISTS artist_song_info ; "
query2 = "DROP TABLE IF EXISTS artist_song_user_info ; "
query3 = "DROP TABLE IF EXISTS user_fullname_info_filtersong ; "
try:
    session.execute(query1)
    session.execute(query2)
    session.execute(query3)
except Exception as e:
    print(e)


### Close the session and cluster connection¶

In [28]:
session.shutdown()
cluster.shutdown()